In [5]:
import pandas as pd
from utils import *
from constant import *

In [6]:
df = pd.read_csv("output6.csv",index_col=0)
df = df[~df["mean"].duplicated()]
# df["recent"] = df.index.str.slice(-1)
df = df[df["count"] > 30]
df.head(3)

,win_ratio,mean,count
foot2-~opentop0-top0-~yz1-top1-opentop1,0.606061,1.021372,33
small_volume-yz0-~yz1-top1-opentop1,0.683333,1.021688,60
small_volume-~opentop0-top0-~yz1-top1-opentop1,0.620690,1.022867,58


In [3]:
df = df.sort("win_ratio").tail(10)

1439

In [4]:
df = df[(df.index == "small_volume-yz0-~opentop1-top1") | (df.index == "~foot1-small_volume-yz0")]

In [7]:
target_factor = "small_volume-yz0-~yz1-top1-opentop1"

In [8]:
valid_keys = remove_invalid(df,target_factor)

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


small_volume-yz0-~yz1-top1-opentop1 ~speedup1-small_volume-~yz1-top1-opentop1


In [11]:
valid_keys

{'~foot1-small_volume-yz0'}

In [16]:
valid_keys

{'jump1-jump2-foot0-~foot1-foot2-small_volume-~opentop1-top1',
 'jump2-small_volume-~yz0-top0-opentop0',
 'jump2-~foot0-small_volume-~yz0-top0-opentop0',
 'jump2-~foot0-~isnew-small_volume-~yz0-top0-opentop0',
 'jump2-~isnew-small_volume-~yz0-top0-opentop0',
 'jump2-~yz0-top0-opentop0-~opentop1-top1',
 'opentop2-~speedup1-jump1-~foot0-~foot1-~opentop1-top1',
 'speedup1-foot0-isnew',
 'speedup1-jump1-foot1-isnew',
 '~foot1-small_volume-yz0',
 '~opentop2-foot0-isnew-small_volume',
 '~opentop2-isnew-small_volume',
 '~opentop2-jump1-foot0-foot2-small_volume',
 '~opentop2-jump1-small_volume-~yz1-top1-opentop1',
 '~opentop2-speedup1-jump1-~speedup2-jump2-~foot0-foot1-small_volume',
 '~opentop2-speedup1-~speedup2-jump2-~foot0-foot1-small_volume',
 '~opentop2-~jump1-~jump2-foot0-small_volume-~opentop0-top0',
 '~opentop2-~jump1-~jump2-foot0-~foot1-small_volume-~opentop0-top0',
 '~opentop2-~jump1-~speedup2-foot0-small_volume-~opentop0-top0',
 '~opentop2-~speedup1-isnew-small_volume',
 '~opentop2

In [8]:
for i in range(2,8):
    sub_df = df[df["recent"] == str(i)]
    valid_keys = remove_invalid(sub_df,i)
    print valid_keys
    print len(valid_keys),len(sub_df),i
    print "--------"

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:15: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


set(['~opentop2-~speedup1-jump1-~jump2-foot1-small_volume-2', 'speedup1-speedup2-jump2-~foot1-~small_volume-~opentop1-top1-2', 'speedup1-~jump1-~foot1-isnew-2', '~opentop2-~speedup1-jump1-speedup2-~foot0-~foot1-~foot2-2', 'opentop2-foot0-~small_volume-2', '~opentop2-~jump1-isnew-small_volume-2', '~speedup1-jump1-jump2-~foot1-~isnew-small_volume-2', 'jump1-isnew-yz1-2'])
8 231 2
--------
set(['~speedup1-speedup2-~foot1-foot2-small_volume-3', 'jump1-~speedup2-jump2-~foot1-foot2-~small_volume-3', '~opentop2-~speedup1-foot1-~foot2-small_volume-~opentop1-top1-3', '~speedup1-~speedup2-foot0-~foot1-~yz1-top1-opentop1-3', '~opentop2-~foot0-~small_volume-~yz0-top0-opentop0-3', '~opentop2-jump2-foot0-foot2-small_volume-3', '~opentop2-speedup1-foot1-foot2-small_volume-3', 'speedup1-~foot2-isnew-small_volume-~opentop1-top1-3', 'opentop2-~speedup1-~foot0-3', 'jump1-speedup2-jump2-~foot0-foot1-3', '~foot1-small_volume-yz0-3'])
11 600 3
--------
set(['jump1-~foot0-~foot2-~isnew-small_volume-~opentop0

In [5]:
def remove_invalid(output_df,target_factor):
    keys = output_df.index
    valid_keys = set(keys)
    for keyA in keys:
        if not keyA in valid_keys:
            continue
        filterA = getFilter(keyA)
#         filter_recent = recent(day,data_df)
        for keyB in keys:
            if not keyB in valid_keys or keyA == keyB:
                continue
            filterB = getFilter(keyB)
            sub_dfA = data_df[filterA & ~filterB & ~yz2]
            sub_dfB = data_df[filterB & ~filterA & ~yz2]
#             print output_df.head(3)
            to_remove,to_break = shouldRemove(keyA,keyB,sub_dfA,sub_dfB,output_df)
            if to_remove is not None:
                if to_remove == target_factor:
                    print keyA,keyB
                    target_factor = keyA if keyB == to_remove else keyB
                valid_keys.remove(to_remove)
            if to_break:
                break            
    return valid_keys

In [6]:
def shouldRemove(keyA,keyB,sub_dfA,sub_dfB,output_df):
    if is_valid(sub_dfA) and not is_valid(sub_dfB):
#         print "B not valid"
        return keyB,False
    elif not is_valid(sub_dfA) and is_valid(sub_dfB):
#         print "A not valid"
        return keyA,True
#     elif not is_valid(sub_dfA) and not is_valid(sub_dfB):
#         if output_df["count"][keyA] > output_df["count"][keyB]:
# #             print "count A > count B"
#             return keyB,False
#         elif output_df["count"][keyA] < output_df["count"][keyB]:
# #             print "count A < count B"
#             return keyA,True
#         else:
#             if keyA.split("-") < keyB.split("-"):
#                 return keyB,False
#             elif keyA.split("-") > keyB.split("-"):
#                 return keyA,True
#             else:
#                 if output_df["win_ratio"][keyA] >= output_df["win_ratio"][keyB]:
#                     return keyB,False
#                 else:
#                     return keyA,True
    else:
        return None,False

In [4]:
def is_valid(sub_df):
    sub_df = sub_df.sort_values("minute")
    sub_df = sub_df.groupby("date").first()
    count = len(sub_df)
    mean = sub_df.change.mean()
    if mean < 1.005 or count < 1:
#         print mean,count
        return False 
    win_df = sub_df[sub_df["change"] > 1.0]
    win_ratio = float(len(win_df))/len(sub_df)
#     print mean,win_ratio
    if count < 10:
        return mean > 1.02 and win_ratio > 0.6
    else:
        return mean > 1.01 and win_ratio > 0.55

In [28]:
def remove_invalid(output_df,day):
    keys = output_df.index
    valid_keys = set(keys)
    for keyA in keys:
        if not keyA in valid_keys:
            continue
        filterA = getFilter(keyA)
        filter_recent = recent(day,data_df)
        for keyB in keys:
            if not keyB in valid_keys or keyA == keyB:
                continue
            print keyA + "--" + keyB
            filterB = getFilter(keyB)
            sub_dfA = data_df[filter_recent & filterA & ~filterB & ~yz2]
            sub_dfB = data_df[filter_recent & filterB & ~filterA & ~yz2]
            to_remove,to_break = shouldRemove(keyA,keyB,sub_dfA,sub_dfB,output_df)
            if to_remove is not None:
                if to_remove == "jump1-~foot1-small_volume-yz0":
                    print keyA,keyB
                valid_keys.remove(to_remove)
            if to_break:
                break            
    return valid_keys

In [7]:
win_ratio_se = pd.Series()
mean_se = pd.Series()
for filters in filterss:
    each_filters = []
    for filter_key in filters:
        filter_a = getFilter(filter_key,True)
        sub_df = data_df[filter_a]
        sub_df = sub_df.sort_values("minute")
        sub_df = sub_df.groupby("date").first()
        count = len(sub_df)
        mean = sub_df.change.mean()
        win_df = sub_df[sub_df["change"] > 1.0]
        win_ratio = float(len(win_df))/len(sub_df)
        win_ratio_se[filter_key] = win_ratio
        mean_se[filter_key] = mean
df = pd.DataFrame()
df["win_ratio"] = win_ratio_se
df["mean"] = mean_se
df.to_csv("output4.csv")

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
